In [ ]:
!wget https://cb.lk/covid_19

In [ ]:
!unzip covid_19

In [ ]:
train_path = "CovidDataset/Train"
val_path = "CovidDataset/Test"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

In [ ]:
#We are building a sequential layers
#The input shape is (224,224) as it is somewhat standard and ImageNet models 
#and a few other models are trained in this size, it is not too big, not too small
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3), activation='relu', input_shape=(224,224,3)))
model.add(Conv2D(64,kernel_size=(3,3), activation='relu'))
#Adding two (3,3) layers instead of one (5,5) layer would increase the non linearity which 
#would eventually increase the ability of the network to learn complex functions
#another benefit is we have lesser number of parameters in two stacked (3,3) layers instead 
#of a single (5,5) layer
model.add(MaxPooling2D(pool_size=(2,2))) 
model.add(Dropout(0.25))

model.add(Conv2D(64,kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128,kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(256,kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics=['accuracy'])



In [ ]:
model.summary()
#As we are go deeper into the network we need to increase the number of channels
#as we go deeper into the network we see more complex features, so increasing the 
#number of channels increases the number of features recognised, since deeper in the network
#we have an increased receptive field.
 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 110, 110, 64)      0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 54, 54, 64)       

In [ ]:
#training phase
#We here are scaling the image, then are introducing some augmentation
#where we have a shear range of some random crops of the image
#and some random zooms of the image too.
#we also include horizontal flipping of the image
# We do not do vertical flipping of the image because our X-rays have 
# Postereoanterior view in which we cannot flip our images.
train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
) 
#We can have class_mode to categorical when we have multi class classification
test_datagen = image.ImageDataGenerator(rescale=1./255)

In [ ]:
train_datagen = train_datagen.flow_from_directory(
    'CovidDataset/Train',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary',
)

Found 224 images belonging to 2 classes.


In [ ]:
train_datagen.class_indices

{'Covid': 0, 'Normal': 1}

In [ ]:
validation_generator = test_datagen.flow_from_directory(
    'CovidDataset/Val',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary'
)

Found 60 images belonging to 2 classes.


In [ ]:
hst = model.fit_generator(
    train_datagen,
    steps_per_epoch=4,
    epochs=10,
    validation_data = validation_generator,
    validation_steps=2
)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
4/4 [==============================] - 8s 2s/step - loss: 0.5194 - accuracy: 0.7734 - val_loss: 0.5520 - val_accuracy: 0.7167
Epoch 2/10
4/4 [==============================] - 7s 2s/step - loss: 0.4228 - accuracy: 0.8125 - val_loss: 0.4447 - val_accuracy: 0.7667
Epoch 3/10
4/4 [==============================] - 7s 2s/step - loss: 0.4485 - accuracy: 0.7891 - val_loss: 0.3321 - val_accuracy: 0.9500
Epoch 4/10
4/4 [==============================] - 7s 2s/step - loss: 0.3789 - accuracy: 0.8594 - val_loss: 0.3090 - val_accuracy: 0.9000
Epoch 5/10
4/4 [==============================] - 7s 2s/step - loss: 0.2679 - accuracy: 0.8672 - val_loss: 0.2050 - val_accuracy: 0.9667
Epoch 6/10
4/4 [==============================] - 7s 2s/step - loss: 0.1971 - accuracy: 0.9297 - val_loss: 0.1798 - val_accuracy: 0.9000
Epoch 7/10
4/4 [==============================] - 7s 2s/step - loss: 0.2894 - accuracy: 0.9141 - val_loss: 0.1665 - val_accuracy: 0.9833
Epoch 8/10
4/4 [=========================

In [ ]:
# Class activation mappings
# Grad CAM technique